## Tutorial on a basic linear classifier

This is a tutorial on the basic structure of using a planar decision boundary to divide a collection of data-points into two classes, by Rajeev Raizada.

Please mail any comments or suggestions to: rajeev dot raizada at gmail dot com

Thank you to Michael Hanke, who first converted the original Matlab into Python, when incorporating this tutorial into PyMVPA: http://www.pymvpa.org/examples/hyperplane_demo.html

I have used his Python version here, with some modifications.

This is a live Jupyter notebook, running on the Google Colab server (for free!). So, you can run the code, make changes to it, and see what happens. You need have a Google login (e.g. a Gmail account) in order to run the code on Google Colab. Before you run the code, Google will show a warning, telling you to check the code below, in order to make sure that it isn't trying to do anything malicious. Don't worry, it isn't! However, it's good practice to check, so please do take a look at the code below, before running it.

If you want to download the notebook, in order to run it on your own computer, then you can get it here: [classification_plane_tutorial.ipynb](http://www.bcs.rochester.edu/people/raizada/Python/classification_plane_tutorial.ipynb) 

#### Suggested exercise: change the code, re-run it, and see what happens

A good exercise is to change the coordinates of some of the input data points, and see how this affects the classifier's resulting decision boundary.

This tutorial aims to be very introductory, showing how a classification task (in this case, deciding whether people are sumo wrestlers or basketball players, based on their height and weight) can be viewed as drawing a decision boundary in a feature space. It shows how to plot the data, calculate the weights of a simple linear classifier, and see how the resulting classifier carves up the feature space into two categories.

First, we'll import some standard libraries that we'll need: NumPy, for doing math, and pyplot for making figures

In [0]:
from matplotlib import pyplot as plt 
import numpy as np

# This next line just makes the fonts bigger. Their default size is too small
plt.rcParams.update({'font.size': 12})

### Two classes: sumo wrestlers and basketball players

Let's look at a toy example: classifying people as either
sumo wrestlers or basketball players, depending on their height and weight.

Let's use height as the first dimension, and weight as the second dimension. Because we are later going to calculate weights for our classifier, we'll call the people's weight "body-weight", so as not to cause confusion between body-weight and classifier-weights!

In [0]:
sumo_wrestlers_height =     [ 4, 2, 2, 3, 4 ]
sumo_wrestlers_bodyweight = [ 8, 6, 2, 5, 7 ]

basketball_players_height =     [ 3, 4, 5, 5, 3 ]
basketball_players_bodyweight = [ 2, 5, 3, 7, 3 ]

Let's plot this:

In [0]:
plt.plot(sumo_wrestlers_height, sumo_wrestlers_bodyweight, 'ro', label="Sumo wrestlers")
plt.plot(basketball_players_height, basketball_players_bodyweight, 'bx',label="Basketball players")
plt.xlim(1, 6)
plt.ylim(0, 12)
plt.xlabel('Height')
plt.ylabel('Body weight')
plt.legend();

Let’s stack up the sumo data on top of the basketball players data, so that it's all part of one data-matrix. Later below, this will make it easier for us to fit our classifier to all of the data at once.

We'll turn the height and weight lists into NumPy arrays, so that we can do math with them, and then we'll transpose them, to turn row vectors into column vectors.

Then we'll use the NumPy command vstack() to stack those two column vectors vertically on top of each other.

In [0]:
# Turn the sumo data into np.arrays, instead of lists
sumo_data = np.array([sumo_wrestlers_height, sumo_wrestlers_bodyweight])

# Transpose to have each row be one person's data, using .T
# Transposing flips the rows into columns (and vice versa)
sumo_data = sumo_data.T

Let's look at the sumo_data numbers and shape, to verify that it is indeed now a matrix, with the first column being height, the second column being weight, and each row being the height-weight info for one person.

To do this, we use the print() command. Before we print out the actual matrix of numbers, we'll also print out the string 'sumo_data', so that we know which variable the numbers are coming from, and then a newline character, '\n', so that the matrix gets printed out starting on its own new line.

In [0]:
print('sumo_data','\n',sumo_data)

Now let's do the same for the basketball players' data

In [0]:
basketball_data = np.array([basketball_players_height, basketball_players_bodyweight])

basketball_data = basketball_data.T

print('basketball_data','\n',basketball_data)


Now we'll stack them all together, to make all_data

In [0]:
all_data = np.vstack((sumo_data, basketball_data))

print('all_data','\n',all_data)

In order to be able to train a classifier on the input vectors, we need to know what the desired output categories are for each one.

Let's set this to be +1 for sumo wrestlers, and -1 for basketball players.

Our height-weight data matrix has five rows for sumo wrestlers, and then another five rows for basketball players. So, we need a desired-output column vector that has five rows of +1, and then five rows of -1.

We can do this using the NumPy command ones(), which makes an array of ones with however many rows and columns we pass in as inputs.


In [0]:
# Let's make an array of 1s, with five rows and one column
sumo_desired_output = np.ones((5,1))

# For the column vector of -1s, we simply pre-multiply by -1
basketball_desired_output = -1 * np.ones((5,1))

# Now we stack them together
all_desired_output = np.vstack([sumo_desired_output,basketball_desired_output])

# Let's look at our vector, to check that it has the right shape.
# Note that the ones get printed as '1.'
# That means that the 1 is being represented as a floating-point number,
# meaning that we can do math with it using decimal places.
print(all_desired_output)


We want to find a linear decision boundary, i.e. simply a straight line, such that all the data points on one side of the line get classified as sumo wrestlers, i.e. get mapped onto the desired output of +1, and all the data points on the other side get classified as basketball players, i.e. get mapped onto the desired output of -1.

The equation for a straight line has this form:

weight_vector * data_coords  -  offset_from_origin = 0

We're not so interested for now in the offset_from_origin term, so we can get rid of that by subtracting the mean from our data, so that it is all centered around the origin.

The means that we want to subtract are the mean-height, i.e. the mean of the first column, and the mean-weight, which is the mean of the second column. 

In Python, we start counting at zero, so the columns are the 0th axis. So, we can subtract the column-means from the data like this:

In [0]:
zero_meaned_data = all_data - all_data.mean(axis=0)

Now, having gotten rid of that annoying offset_from_origin term, we want to find a weight vector which gives us the best solution that we can find to this equation:

zero_meaned_data * weights = all_desired_output

However, there is no such perfect set of weights that will yield exactly all_desired_output as the output. We can only get a best fit, such that:

zero_meaned_data * weights = all_desired_output + error

where the error term is as small as possible.

Note that our equation:

zero_meaned_data * weights = all_desired_output

has exactly the same form as the equation
from the tutorial code in [design_matrix_tutorial.ipynb](https://colab.research.google.com/drive/1JxT6UsrLRbKsY-QG-1w--lUUE3UD26PK#sandboxMode=true)

which is:

Design matrix * sensitivity vector = Voxel response

## Computing the classifier's weights

The way we solve the equation is exactly the same, too. If we could find a matrix-inverse of the data matrix, then we could pre-multiply both sides by that inverse, and that would give us the weights:

inv(zero_meaned_data) * zero_meaned_data * weights = inv(zero_meaned_data) * all_desired_output


The inv(zero_meaned_data) and zero_meaned_data terms on the left would cancel each other out, and we would be left with:

weights = inv(zero_meaned_data) * all_desired_output

However, unfortunately there will in general not exist any matrix-inverse of the data matrix zero_meaned_data.
Only square matrices have inverses, and not even all of them do. Luckily, however, we can use something that plays a similar role, called a pseudo-inverse. In NumPy, this is given by the command pinv.

The pseudo-inverse won't give us a perfect solution to the equation

zero_meaned_data * weights = all_desired_output

but it will give us the best approximate solution, which is what we want.

So, instead of

weights = inv(zero_meaned_data) * all_desired_output

we have this equation:

weights = pinv(zero_meaned_data) * all_desired_output

If we type np.linalg.pinv instead of pinv, so that we tell Python that the pinv function is part of NumPy's linear algebra library, then we can literally execute that equation as the command to give us our best-fit weights. One small detail is the Python symbol for matrix-multiplication is @, rather than *.


In [0]:
weights = np.linalg.pinv(zero_meaned_data) @ all_desired_output

Let's have a look at this weight-vector. 

It has two elements: the first is the weight which gets used to multiply the first dimension (height), and the second is the weight that multiplies the second dimension (body-weight).

You see now why we added body- to body-weight. Otherwise we'd be saying that the second element of weights gets used to multiply the weights, which would be confusing!

In [0]:
print('weights','\n',weights)

Note that the first weight is negative. Recall that this corresponds to the height-dimension, and that sumo wrestlers got given the class-label +1.

So, a negative weight on the height dimension means that the more height you have, the **less** likely you are to be a sumo wrestler. And that makes sense, because sumo wrestlers tend to be less tall than basketball players.

Similarly, the second weight is positive. This corresponds to the body-weight dimension, so its positive value means that the more weight you have, the **more** likely you are to be a sumo wrestler. Again, that makes sense.

## Seeing how well the classifier performs

Ok, let's see whether the classifier actually puts these data points into the correct categories. 

Note that if we were actually trying to do a real classification task, then we would split the data into training and test sets. This is just a toy example, so we're not going to do that here: we'll simply see how well the classifier performs on all the data, after having been trianed on all of that same data. In real life, this would not be a useful thing to do!

That aside, let's go ahead anyway. First, let's look at the raw weighted output, before it gets thresholded into two distinct classes. To do this, we simply matrix-multiply the zero-meaned data by our freshly calculated weights. Recall that this uses the @ sign in Python:

In [0]:
weighted_output = zero_meaned_data @ weights

print('weighted_output','\n',weighted_output)

We can see right away that the first five weighted outputs, which correspond to the sumo wrestlers and which are meant to get mapped to +1, are all positive. So, even though they don't get mapped to that exact desired-output, they do end up on the correct side of zero.

Similarly, the next five weighted outputs are all meant to get mapped to -1, for basketball players, and they are all negative.

To threshold the weighted output, so as to yield the output category-decision, we map to +1 if the weighted output is greater than 0, and -1 if it is less than zero.

The easiest way to map positive numbers to +1
and negative numbers to -1 is by first mapping them to 1 and 0 by the inequality-test

(weighted_output_Z>0)

and then turning 1 and 0 into +1 and -1

by multipling by 2 and subtracting 1.

In [0]:
decision_output = 2*(weighted_output>0) - 1

print('decision_output','\n',decision_output)

This confirms that we have 1s and -1s in all the desired places. To turn this into a percentage-correct score, we can assign a 1 to every correct output, a zero to every incorrect output, and calculate the average. Note that we use a double-equals sign, ==, to test if two things are equal to each other or not. A single equals sign, in contrast, assigns the value of the thing to the right of the equals sign to the variable on the left hand side of the equals sign. 

In [0]:
output_is_correct = (decision_output == all_desired_output)
num_correct = sum(output_is_correct)

percentage_correct = 100 * num_correct/len(decision_output)

print('The classifier got', percentage_correct[0], 'percent correct on the training set')

## Visualising how the classifier carves up the input space

Let's have a look at how these weights carve up the input space.

A useful Python command for making grids of points
which span a particular 2D space is called "meshgrid". We'll make a vector of coordinates going from -7 to 7, split up into 100 pieces, and we'll make a meshgrid out of that.

In [0]:
coords_vec = np.linspace(-7, 7, 100)

input_space_X, input_space_Y = np.meshgrid(coords_vec, coords_vec)

weighted_output_Z = input_space_X * weights[0] + input_space_Y * weights[1]

In [0]:
decision_output_Z = 2*(weighted_output_Z>0) - 1

Let's visualise this decision output.

We'll show the output surface as colour-values, using the Matplotlib command pcolor(). In order to make the colours match up nicely with the ones we originally chose for our first plot, we'll use a colormap called "coolwarm", which goes from blue to red. And in order to make this coloured surface not too visually obtrusive, we'll make it fairly transparent, using what is called an alpha-channel value.

In [0]:
plt.pcolor(input_space_X, input_space_Y, decision_output_Z,\
           cmap='coolwarm',alpha=0.1)

plt.xlabel('De-meaned height')
plt.ylabel('De-meaned body-weight')
plt.title('Decision output')
plt.xlim(-2.5, 2.5)
plt.ylim(-4, 7);

Now let's show the original data points plotted on it, too. 

Because we classified the zero-meaned data, we'll pull from that data array in order to get our plot points. The first five points are our sumo wrestlers, and the next five are our basketball players.

Note a slight weirdness in the way Python counts: to get the first five of something, we start at zero (we already knew that), but we stop at 5, not at 4, as one might have thought. The second number means "stop before you get to this one". So, we access the first five entries with [0,5], and the next five with [5,10]. That may look like we are getting the 5th entry twice, but actually we aren't!

In [0]:
plt.pcolor(input_space_X, input_space_Y, decision_output_Z,\
           cmap='coolwarm',alpha=0.1)

# First we'll plot the first five data points: the sumo wrestlers
plt.plot(zero_meaned_data[0:5,0],zero_meaned_data[0:5,1],\
         'ro', label="Sumo wrestlers")

# Next, we'll plot the next five data points: the basketball players
plt.plot(zero_meaned_data[5:10,0],zero_meaned_data[5:10,1],\
         'bx', label="Basketball players")
                  
plt.xlabel('De-meaned height')
plt.ylabel('De-meaned body-weight')
plt.title('Decision output, and the data points to be classified')
plt.legend()

plt.xlim(-2.5, 2.5)
plt.ylim(-4, 7);
